In [1]:
import os
import re
import pandas as pd
import pickle
import collections
from collections import defaultdict
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen

from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

c:\programdata\miniconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# define Python user-defined exceptions
class Error(Exception):
    """Base class for other exceptions"""
    pass


class ErrorInReviewScraping(Error):
    """Raised when scraping commnets"""
    pass

In [91]:
def scrap_product_comments(soup):
    
    try:
        
        #if comments exist:
        if soup.find_all("div", {"id": "comment_list_area"}) != []:
            
            #comments list area
            comment_list_area = BeautifulSoup(str(soup.find_all("div", {"id": "comment_list_area"})), "html.parser")
            comment_total_cnt = int(comment_list_area.find("em", {"id": "gCommentTotalCnt"}).text.replace(',', ''))

            #comment table area
            comment_table_area = BeautifulSoup(str(soup.find_all("table", {"summary": "전체 상품평 보여주는 표"})), "html.parser")
            
            #rating(necessary)
            ratings = BeautifulSoup(str(comment_table_area.find_all("span", {"class": "sp_cdtl cdtl_cmt_star"})), "html.parser")
            rating = [x.text for x in ratings.find_all("em")]
            
            #user(necessary)
            users = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "user"})), "html.parser")
            user = [x.text.strip() for x in users.find_all("div", {"class":"in"})]
            
            #review date(neccessary)
            dates = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "date"})), "html.parser")
            date = [x.text.strip() for x in dates.find_all("div", {"class":"in"})]

            #comment row
            comment_row = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "desc_txt"})), "html.parser")
            
            #review0_typ_month(option)
            review_typ_month = [x.find_all("span", {"class": "review_typ typ_month"}) for x in comment_row.find_all("td", {"class": "desc_txt"})]
            for i in range(len(review_typ_month)):
                if review_typ_month[i] != []:
                    review_typ_month[i] = '한달사용리뷰'
                else:
                    review_typ_month[i] = None
                    
            #purchase option(option)
            cdtl_cmt_tx2 = BeautifulSoup(str(comment_row.find_all("div", {"class": "cdtl_cmt_tx2"})), "html.parser")
            purchase_option = [x.find_all("div", {"class": "cdtl_cmt_tx2"}) for x in comment_row.find_all("td", {"class": "desc_txt"})]

            for i in range(len(purchase_option)):
                if purchase_option[i] != []:
                    purchase_option[i] = cdtl_cmt_tx2.find_all("div")[i].text
                else:
                    purchase_option[i] = None
                    
            #comment view for scraping content of review
            comment_view = BeautifulSoup(str(comment_table_area.find_all("tr", {"class": "cdtl_cmt_view"})), "html.parser")
            
            #evaluation items and content of evaluation (option)
            review_eval = BeautifulSoup(str(comment_view.find_all("ul", {"class":"review_eval"})), "html.parser")
            evaluation_item = []
            evaluation = []
            for i in range(len(comment_view.find_all("tr", {"class": "cdtl_cmt_view"}))):
                review_eval = BeautifulSoup(str(comment_view.find_all("tr", {"class": "cdtl_cmt_view"})[i].find_all("ul", {"class":"review_eval"})), "html.parser")
                evaluation_item.append([x.text for x in review_eval.find_all("h5")]) #고객평가항목
                evaluation.append([x.text for x in review_eval.find_all("span")]) #고객평가항목평가
                evaluation_item = [elm if elm!= [] else None for elm in evaluation_item]
                evaluation = [elm if elm!= [] else None for elm in evaluation]
            
            #review description(neccesary)
            comment_desc = [x.text.replace('\n', '') for x in comment_view.find_all("div", {"class": "desc"})]

            return rating, user, date, review_typ_month, evaluation_item, evaluation, comment_desc, purchase_option
        
        else:
            return None 
        
    except Exception as e:
        raise ErrorInReviewScraping('error occurs during scraping comments: ', e)
        
#get comment naviagtion area
def current_comment_navi_area(soup):
    return soup.find("div", {"id": "comment_navi_area"})


#scrap comment from first page to last page
def scrap_product_comment_all(soup):
    
    try:
        #comments in first page
        comments = scrap_product_comment_soup
        
        #if comments exist in first page
        if comments != None:
            rating, user, review_date, review_typ_month, evaluation_item, evaluation, comment_desc, user_purchase_option = scrap_product_comments(soup)
            
            #comment_nevigation_area
            comment_navi_area = current_comment_navi_area(soup)
            
            try:
                if comment_navi_area.find("a", {"class": "btn_last"}):
                    last_comment_page_num = int(re.findall("fn_GoCommentPage\('(\d*?)\\'\).*?", str(comment_navi_area.find("a", {"class": "btn_last"})))[0])
                else:
                    last_comment_page_num = int(comment_navi_area.find_all("a")[-1].text)
            except:
                last_comment_page_num = 1            
        
            #scrap comment from first page to last page
            for i in range(last_comment_page_num-1):

                prev_btn_cnt = len(re.findall('onclick="fn_GoCommentPage', str(comment_navi_area).split('<strong title="현재위치"')[0])) + 1
                wd.find_element_by_css_selector("#comment_navi_area > a:nth-child({})".format(prev_btn_cnt+1)).click()
                time.sleep(5)
                
                #re-parsing
                html = wd.page_source
                time.sleep(3)
                soup = BeautifulSoup(html, "html.parser")
                comment_navi_area = BeautifulSoup(str(soup.find("div", {"id": "comment_navi_area"})), "html.parser")

                #scrap comment in current page
                rating_add, user_add, review_date_add, review_typ_month_add, evaluation_item_add, evaluation_add, comment_desc_add, user_purchase_option_add = scrap_product_comments(soup)

                #merge comment data with data scrapped at previous pages
                rating = rating + rating_add
                user = user + user_add
                review_date = review_date + review_date_add
                review_typ_month = review_typ_month + review_typ_month_add
                evaluation_item = evaluation_item + evaluation_item_add
                evaluation = evaluation + evaluation_add
                comment_desc = comment_desc + comment_desc_add  
                user_purchase_option = user_purchase_option + user_purchase_option_add

        #no comment exists in first page
        else:
            rating = None
            user = None
            review_date = None
            review_typ_month = None
            evaluation_item = None
            evaluation = None
            comment_desc = None
            user_purchase_option = None

        return rating, user, review_date, review_typ_month, evaluation_item, evaluation, comment_desc, user_purchase_option
    
    except Exception as e:
        raise ErrorInReviewScraping('error occurs during scraping comments: ', e)
        
        
#scrap product total information
def scrap_ssg_product_info(brand_id, product_id):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    socket.setdefaulttimeout(30)
    
    while True: #to loop infinitely when HTTPConnectionPool
        
        try:
            wd.get('http://www.ssg.com/item/itemView.ssg?itemId={}'.format(product_id))
            wd.maximize_window()
            time.sleep(2)

            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")


            #product name(necessary)
            prd_name = soup.find("h2", {"class": "cdtl_info_tit"}).text
            prd_price = soup.find("em", {"class": "ssg_price"}).text

            try:
                model_num = soup.find("p", {"class": "cdtl_model_num"}).text.split(': ')[-1]
            except:
                model_num = None

            
            #product essential information(necessary)
            cdtl_cont_info = BeautifulSoup(str(soup.find_all("div", {"class": "cdtl_tbl ty2"})), "html.parser")  #class 수정
            product__essential_info_items = [x.text for x in cdtl_cont_info.find_all("th", {"scope": "row"})]
            product__essential_info = [x.text for x in cdtl_cont_info.find_all("td")]

            #product option
            if soup.find("div", {"id": "_ordOpt_area"}) != None:
                opt = BeautifulSoup(str(soup.find_all("div", {"id": "_ordOpt_area"})), "html.parser")
                opt_lst = BeautifulSoup(str(opt.find_all("ul", {"class", "cdtl_select_lst _drop_list"})), "html.parser")
                options = [x.text for x in opt_lst.find_all("span", {"class": "txt"})]  
            else:
                options = None

            #category
            if soup.find("span", {"class": "cdtl_sel_item"}) != None:
                select_item = BeautifulSoup(str(soup.find_all("span", {"class": "cdtl_sel_item"})), "html.parser")
                item_selection = [x.text.strip() for x in select_item.find_all("span", {"class": "cdtl_txt"})]
            else:
                item_selection = None
            
            #if selection list for comments exists
            if soup.find("div", {"class": "cdtl_info_sel"}) != None:
                cdtl_info_sel = BeautifulSoup(str(soup.find("div", {"class": "cdtl_info_sel"})), "html.parser")
                cdtl_info_sel_len = len(cdtl_info_sel.find_all("option")) - 1

                rating = []
                user = []
                review_date = []
                review_typ_month = []
                evaluation_item = []
                evaluation = []
                comment_desc = []
                user_purchase_option = []
                
                #loop to scrap comments of every item in selection list
                for i in range(1, cdtl_info_sel_len+1):
                    sel_arrow = wd.find_element_by_css_selector("#dealItemCommentTab > div.cdtl_info_sel > div > a > span.sp_chd.sel_arrow") #arrow to view selection list
                    time.sleep(3)
                    actions = ActionChains(wd)
                    time.sleep(2)
                    actions.move_to_element(sel_arrow).perform() #scroll to selection arrow to click it
                    time.sleep(2)

                    wd.find_element_by_css_selector("#dealItemCommentTab > div.cdtl_info_sel > div > a > span.sp_chd.sel_arrow").click() #click selection arrwo to view selection list
                    time.sleep(2)
                    wd.find_element_by_css_selector("#dealItemCommentTab > div.cdtl_info_sel > div > div > ul > li:nth-child({})".format(i)).click() #select item
                    time.sleep(3)

                    html = wd.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    
                    #scrap all comments in all pages for each item selection
                    rating_opt, user_opt, review_date_opt, review_typ_month_opt, evaluation_item_opt, evaluation_opt, comment_desc_opt, user_purchase_option_opt = scrap_product_comment_all(soup) #각 상품의 리뷰 페이지 돌기
                    
                    rating.append(rating_opt)
                    user.append(user_opt)
                    review_date.append(review_date_opt)
                    review_typ_month.append(review_typ_month_opt)
                    evaluation_item.append(evaluation_item_opt)
                    evaluation.append(evaluation_opt)
                    comment_desc.append(comment_desc_opt)
                    user_purchase_option.append(user_purchase_option_opt)

            #no selection list
            else:
                rating, user, review_date, review_typ_month, evaluation_item, evaluation, comment_desc, user_purchase_option = scrap_product_comment_all(soup)

            wd.close()

            prd_info = {
                "brand_id": brand_id,
                "brand_name": None,
                "product_id": product_id,
                "product_name": prd_name,
                "product_price": prd_price,
                "model_number": model_num,
                "product__essential_info_items": product__essential_info_items,
                "product__essential_info": product__essential_info,
                "option": options,
                "category": category,
                'rating': rating,
                'user': user,
                'review_date': review_date,
                'review_typ_month': review_typ_month,
                'evaluation_item': evaluation_item,
                'evaluation': evaluation,
                'comment_desc': comment_desc,
                'user_purchase_option': user_purchase_option
            }

            return prd_info

        except urllib3.exceptions.ReadTimeoutError:
            time.sleep(5)
            continue

        except ErrorInReviewScraping as re_e: #error raised during scraping comments
            print("제품 리뷰 정보 수집에서 에러가 발생했습니다.", re_e)
            return None

        except Exception as e:
            print("제품 객체 정보 수집에서 에러가 발생했습니다.", e)
            return None

In [93]:
ssg_prd_info = pd.DataFrame(columns = ['brand_id',
                                      'brand_name', 
                                      'product_id',
                                      'product_name',
                                      'product_price',
                                      'model_number',
                                        'product__essential_info_items',
                                        'product__essential_info',
                                       'option',
                                       'category',
                                      'rating',
                                      'user',
                                      'review_date',
                                      'review_typ_month',
                                      'evaluation_item',
                                      'evaluation',
                                      'comment_desc',
                                      'user_purchase_option'])

In [ ]:
idx = 0
for brd_id in tqdm(brd_w_prd.keys()):
    for prd_id in tqdm(brd_w_prd[brd_id]):
        prd_info = scrap_ssg_product_info(brd_id, prd_id)
        if prd_info != None:
            ssg_prd_info.loc[idx] = prd_info
            idx += 1

Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


...



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
